# Regridding demo datesets using regrid_to_healpix
Before starting this notebook, Create temporaly small data set using the noteoobk 'prep_regrid.ipynb'
To install the regrid_to_healpix tool, do

```pip install "git+https://github.com/EOPF-DGGS/regrid_to_healpix.git"```

In [1]:

import xarray as xr
import numpy as np

In [2]:

#small_ds=xr.open_zarr('/Users/todaka/data/RIOMAR/very_small.zarr')
small_ds=xr.open_zarr('/Users/todaka/data/RIOMAR/small.zarr')

small_ds

<xarray.Dataset> Size: 2MB
Dimensions:       (time_counter: 2, s_rho: 40, y_rho: 50, x_rho: 68)
Coordinates:
  * time_counter  (time_counter) datetime64[ns] 16B 2023-12-01T00:48:10 2023-...
    time_instant  (time_counter) datetime64[ns] 16B dask.array<chunksize=(2,), meta=np.ndarray>
  * s_rho         (s_rho) float32 160B -0.9875 -0.9625 ... -0.0375 -0.0125
  * y_rho         (y_rho) float32 200B 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
  * x_rho         (x_rho) float32 272B 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
    mask          (y_rho, x_rho) bool 3kB dask.array<chunksize=(50, 68), meta=np.ndarray>
    nav_lat_rho   (y_rho, x_rho) float32 14kB dask.array<chunksize=(50, 68), meta=np.ndarray>
    nav_lon_rho   (y_rho, x_rho) float32 14kB dask.array<chunksize=(50, 68), meta=np.ndarray>
Data variables:
    salt          (time_counter, s_rho, y_rho, x_rho) float32 1MB dask.array<chunksize=(1, 40, 50, 68), meta=np.ndarray>
    temp          (time_counter, s_rho, y_rho, x_rho) float32 1MB dask.array<chunksize=(1, 40, 50, 68), meta=np.ndarray>
    zeta          (time_counter, y_rho, x_rho) float32 27kB dask.array<chunksize=(1, 50, 68), meta=np.ndarray>
Attributes: (12/42)
    name:           GAMAR_GLORYS_1h_inst
    description:    Created by xios
    Conventions:    CF-1.6
    title:          GAMAR_GLORYS
    rst_file:       croco_rst.nc
    grd_file:       croco_grd.nc
    ...             ...
    gamma2_expl:    Slipperiness parameter
    x_sponge:       0.0
    v_sponge:       0.0
    sponge_expl:    Sponge parameters : extent (m) & viscosity (m2.s-1)
    SRCS:           main.F step.F read_inp.F timers_roms.F init_scalars.F ini...
    CPP-options:    REGIONAL GAMAR MPI TIDES OBC_WEST OBC_NORTH XIOS USE_CALE...

In [3]:
import hvplot.xarray
small_ds.temp.isel(s_rho=0).hvplot.quadmesh(y='nav_lat_rho', x='nav_lon_rho',  ylim=(46.9, 47.5))#,coastline=True)#),geo=True,datashader=False)#,coastline=True)


:DynamicMap   [time_counter]
   :QuadMesh   [nav_lon_rho,nav_lat_rho]   (potential temperature)

In [4]:
tmp = small_ds[["temp", "nav_lon_rho", "nav_lat_rho"]].isel(s_rho=0).stack(point=("y_rho","x_rho"))
tmp = tmp.dropna(dim="point").compute()  # drops points where any variable is NaN
lon = tmp["nav_lon_rho"].values.astype(np.float64)
lat = tmp["nav_lat_rho"].values.astype(np.float64)
data= tmp.temp.values.astype(np.float64)
data,lon,lat

(array([[15.17735863, 15.0550642 , 14.06368542, ..., 14.3829155 ,
         14.35231686, 12.59215355],
        [15.17770195, 15.07956028, 14.09893703, ..., 14.38112736,
         14.37431145, 12.97036171]], shape=(2, 2193)),
 array([-2.82666659, -2.73333335, -2.5333333 , ..., -2.75999999,
        -2.65333343, -2.55999994], shape=(2193,)),
 array([46.9436264 , 46.9436264 , 46.9436264 , ..., 47.38785172,
        47.38785172, 47.38785172], shape=(2193,)))

## Here we chose one simple method, but this should be refined with RIOMAR project
@jmdelouis @pablo

In [5]:
device = "cpu"
from regrid_to_healpix.regrid_to_healpix_bilinear import Set
#nr = Set(lon_deg=lon, lat_deg=lat, level=level, device=device, threshold=0.7, ellipsoid="WGS84") 
level = 13
nr = Set(lon_deg=lon, lat_deg=lat,level=level, device=device, threshold=0.5, ellipsoid="WGS84")
#from regrid_to_healpix.regrid_to_healpix_GEN import Set
#nr = Set(lon_deg=lon, lat_deg=lat,  Npt = 2, level=level, device=device, ellipsoid="WGS84") 
#from regrid_to_healpix.regrid_to_healpix_nearest import Set
#level = 12
#nr = Set(lon_deg=lon, lat_deg=lat, level=level, device=device, ellipsoid="WGS84") 

#from regrid_to_healpix.regrid_to_healpix_psf import Set
#level = 13
#nr = Set(lon_deg=lon, lat_deg=lat, level=level, device=device, threshold=0.7, ellipsoid="WGS84") 


/Users/todaka/python/git/regrid_to_healpix/regrid_to_healpix/regrid_to_healpix_bilinear.py:74: UserWarning: Sparse CSR tensor support is in beta state. If you miss a functionality in the sparse tensor support, please submit a feature request to https://github.com/pytorch/pytorch/issues. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/aten/src/ATen/SparseCsrTensorImpl.cpp:51.)
  self.M  = M_coo.to_sparse_csr()


In [6]:

# puis:
data_healpix=nr.transform(data,lam=0.1)  # selon ton API
#data_healpix_invert = nr.invert(data_healpix)
cell_ids = nr.get_cell_ids()
#hval.shape#,rval.shape
#cell_ids , data_healpix.shape, data_healpix_invert.shape

In [7]:
import xdggs
def make_xdggs_dataarray_from_cell_ids(data, cell_ids, level=15,  name="da",):
    """
    Build an xdggs-compatible DataArray:
      - dim/coord: 'cell_ids'
      - values: factor (same for all cells)
      - coord attrs required by xdggs
    """
    cell_ids = np.asarray(cell_ids, dtype=np.int64)

    da = xr.DataArray(
        data,
        dims=("cell_ids",),
        coords={"cell_ids": ("cell_ids", cell_ids)},
        name=name,
    )

    da["cell_ids"].attrs.update(
        {
            "grid_name": "healpix",
            "level": int(level),
            "indexing_scheme": "nested",
            "ellipsoid":"WGS84"
        }
    )
    return da.pipe(xdggs.decode).dggs.assign_latlon_coords()
make_xdggs_dataarray_from_cell_ids(data_healpix, cell_ids,level=level).dggs.explore(alpha=0.3)

ValueError: different number of dimensions on data and dims: 2 vs 1

In [8]:
data_healpix.max() - data.max() , data_healpix.min() - data.min(), data_healpix.mean() - data.mean(),

(np.float64(-0.005251869245245899),
 np.float64(0.0),
 np.float64(-0.004745451393270983))

In [ ]:
da